In [1]:
import pandas as pd

# Load the dataset with only the necessary columns
selected_columns = ['Severity', 'Description', 'Street', 'City', 'State', 
                    'Country', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)',
                    'Pressure(in)', 'Visibility(mi)', 'Wind_Direction', 
                    'Wind_Speed(mph)', 'Weather_Condition']

df = pd.read_csv('E:/Master_thesis_REV/Datasets/Dataset_US_accidents/archive/US_Accidents_March23.csv', usecols=selected_columns, encoding='utf-8')

# Display the first few rows to check the data
print(df.head())


   Severity                                        Description  \
0         3  Right lane blocked due to accident on I-70 Eas...   
1         2  Accident on Brice Rd at Tussing Rd. Expect del...   
2         2  Accident on OH-32 State Route 32 Westbound at ...   
3         3  Accident on I-75 Southbound at Exits 52 52B US...   
4         2  Accident on McEwen Rd at OH-725 Miamisburg Cen...   

                      Street          City State Country  Temperature(F)  \
0                     I-70 E        Dayton    OH      US            36.9   
1                   Brice Rd  Reynoldsburg    OH      US            37.9   
2             State Route 32  Williamsburg    OH      US            36.0   
3                     I-75 S        Dayton    OH      US            35.1   
4  Miamisburg Centerville Rd        Dayton    OH      US            36.0   

   Wind_Chill(F)  Humidity(%)  Pressure(in)  Visibility(mi) Wind_Direction  \
0            NaN         91.0         29.68            10.0         

In [2]:
# Handle missing values - example with 'Wind_Speed(mph)'
df['Wind_Speed(mph)'].fillna(df['Wind_Speed(mph)'].mean(), inplace=True)

# Frequency encoding for categorical columns
categorical_columns = ['Description', 'Street', 'City', 'State', 'Country', 'Weather_Condition', 'Wind_Direction']

for col in categorical_columns:
    freq = df[col].value_counts()
    df[col] = df[col].map(freq)

# Drop any remaining rows with missing values if necessary
df = df.dropna()

# Display the first few rows after preprocessing
print(df.head())


c:\Users\manpr\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\completerlib.py:120: UserWarning: using rootmodules_cache requires you to install the `pickleshare` library.
  rootmodules_cache = ip.db.get('rootmodules_cache', {})



Caching the list of root modules, please wait!
(This will only be done once - type '%rehashx' to reset cache!)



c:\Users\manpr\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\completerlib.py:146: UserWarning: using rootmodules_cache requires you to install the `pickleshare` library.
  ip.db['rootmodules_cache'] = rootmodules_cache


   Severity  Description   Street     City   State  Country  Temperature(F)  \
2         2          1.0    252.0   3010.0  118115  7728394            36.0   
3         3          5.0  27546.0  24572.0  118115  7728394            35.1   
4         2          2.0    729.0  24572.0  118115  7728394            36.0   
5         3          1.0    104.0   1073.0  118115  7728394            37.9   
6         2          1.0     26.0  24572.0  118115  7728394            34.0   

   Wind_Chill(F)  Humidity(%)  Pressure(in)  Visibility(mi)  Wind_Direction  \
2           33.3        100.0         29.67            10.0        364470.0   
3           31.0         96.0         29.64             9.0        364470.0   
4           33.3         89.0         29.65             6.0        364470.0   
5           35.5         97.0         29.63             7.0        384840.0   
6           31.0        100.0         29.66             7.0        353806.0   

   Wind_Speed(mph)  Weather_Condition  
2         

In [3]:
# Define features (X) and target (y)
X = df.drop(columns=['Severity'])
y = df['Severity']

# Display the shapes of X and y to confirm
print(f'X shape: {X.shape}, y shape: {y.shape}')


X shape: (5676313, 13), y shape: (5676313,)


In [4]:
from sklearn.model_selection import train_test_split

# Function to sample data
def sample_data(X, y, sample_size):
    if isinstance(sample_size, float):
        if 0 < sample_size < 1.0:
            return train_test_split(X, y, test_size=0.2, train_size=sample_size, random_state=42)
        elif sample_size == 1.0:
            return train_test_split(X, y, test_size=0.001, random_state=42)
        else:
            raise ValueError("sample_size as float must be in the range (0.0, 1.0) or equal to 1.0.")
    elif isinstance(sample_size, int):
        if sample_size > len(X):
            raise ValueError(f"sample_size {sample_size} exceeds the number of available samples {len(X)}.")
        sampled_X = X.sample(n=sample_size, random_state=42)
        sampled_y = y.loc[sampled_X.index]
        return train_test_split(sampled_X, sampled_y, test_size=0.2, random_state=42)
    else:
        raise ValueError("sample_size must be a float or an integer")


In [5]:
import os

os.environ["LOKY_MAX_CPU_COUNT"] = "12"

In [7]:
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import numpy as np
import time
import psutil


# Function to calculate and return metrics
def calculate_metrics(X_train, X_test, y_train, y_test):
    lgb_model = lgb.LGBMRegressor(random_state=42)

    # Define hyperparameters for RandomizedSearchCV
    param_distributions = {
        'n_estimators': [50, 100],  # Number of boosting stages
        'learning_rate': [0.01, 0.05, 0.1],  # Learning rate
        'max_depth': [5, 10],  # Slightly reduced depth to prevent overfitting and speed up training
        'num_leaves': [31, 50],  # Number of leaves in each tree
        'min_child_weight': [0.001, 0.01, 0.1, 1.0],  # Min hessian (regularization)
        'subsample': [0.6, 0.8],  # Subsample ratio of the training instance
        'colsample_bytree': [0.6, 0.8],  # Subsample ratio of columns when constructing each tree
        'force_row_wise': [True]  # Memory-efficient processing
    }

    random_search = RandomizedSearchCV(lgb_model, param_distributions, n_iter=20, cv=3, scoring='neg_mean_squared_error', random_state=42, n_jobs=-1, verbose=2)

    start_time = time.time()
    start_cpu = psutil.cpu_percent(interval=None)
    random_search.fit(X_train, y_train)
    end_cpu = psutil.cpu_percent(interval=None)
    end_time = time.time()

    # Calculate time and CPU usage
    execution_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    y_pred = random_search.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    # Calculate the range of the target variable
    target_range = y_train.max() - y_train.min()

    # Calculate normalized RMSE (nRMSE)
    nrmse = rmse / target_range
    
    memory_usage_MB = X_train.memory_usage(deep=True).sum() / (1024 ** 2)
    normalized_time = execution_time / memory_usage_MB
    
    return {
        'RMSE': rmse,
        'MAPE': mape,
        'R2': r2,
        'MSE': mse,
        'nRMSE': nrmse,  # Normalized RMSE
        'Execution Time (Raw)': execution_time,  # Raw execution time
        'Normalized Time (s/MB)': normalized_time,  # Normalized execution time
        'Average CPU Usage': avg_cpu_usage
    }



In [8]:
os.environ["LOKY_MAX_CPU_COUNT"] = "12"
# Define sample sizes
sample_sizes = [1.0, 0.5, 0.25, 0.125, 100, 1000, 10000, 100000]

# Initialize metrics storage
metrics_list = []
total_execution_time = 0
total_cpu_usage = 0
total_memory_usage_MB = 0

# Loop through each sample size
for size in sample_sizes:
    try:
        X_train_sample, X_test_sample, y_train_sample, y_test_sample = sample_data(X, y, size)
        metrics = calculate_metrics(X_train_sample, X_test_sample, y_train_sample, y_test_sample)
        metrics['Sample Size'] = size
        metrics_list.append(metrics)

        # Accumulate total metrics
        total_execution_time += metrics['Execution Time (Raw)']
        total_cpu_usage += metrics['Average CPU Usage']
        total_memory_usage_MB += X_train_sample.memory_usage(deep=True).sum() / (1024 ** 2)

        print(f"Metrics for sample size {size}:")
        for key, value in metrics.items():
            print(f"{key}: {value}")
        print("-" * 50)

    except Exception as e:
        print(f"An error occurred for sample size {size}: {e}")


Fitting 3 folds for each of 20 candidates, totalling 60 fits


Exception in thread Thread-11 (_readerthread):
Traceback (most recent call last):
  File "c:\Users\manpr\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1052, in _bootstrap_inner
    self.run()
  File "c:\Users\manpr\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 989, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\manpr\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 1597, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
  File "c:\Users\manpr\AppData\Local\Programs\Python\Python312\Lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 3: character maps to <undefined>
c:\Users\manpr\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarni

[LightGBM] [Info] Total Bins 1715
[LightGBM] [Info] Number of data points in the train set: 5670636, number of used features: 13
[LightGBM] [Info] Start training from score 2.161502
Metrics for sample size 1.0:
RMSE: 0.4168793019039398
MAPE: 0.1079958209872158
R2: 0.18258090974874086
MSE: 0.1737883523559162
nRMSE: 0.13895976730131326
Execution Time (Raw): 665.410578250885
Normalized Time (s/MB): 1.0986005715924734
Average CPU Usage: 50.5
Sample Size: 1.0
--------------------------------------------------
Fitting 3 folds for each of 20 candidates, totalling 60 fits
[LightGBM] [Info] Total Bins 1717
[LightGBM] [Info] Number of data points in the train set: 2838156, number of used features: 12
[LightGBM] [Info] Start training from score 2.161454
Metrics for sample size 0.5:
RMSE: 0.41389702317218735
MAPE: 0.10490519715582049
R2: 0.1777854124957221
MSE: 0.1713107457907982
nRMSE: 0.13796567439072913
Execution Time (Raw): 262.66804671287537
Normalized Time (s/MB): 0.86646868647787
Average CP

In [9]:
# Convert metrics to DataFrame
metrics_df = pd.DataFrame(metrics_list)

# Calculate total metrics
total_avg_cpu_usage = total_cpu_usage / len(sample_sizes)
normalized_total_time = total_execution_time / total_memory_usage_MB

# Convert total execution time to minutes and seconds
total_minutes = int(total_execution_time // 60)
total_seconds = total_execution_time % 60

# Display total metrics
print(f"Total Execution Time for Entire Process (Raw): {total_minutes} minutes and {total_seconds:.2f} seconds")
print(f"Total Normalized Execution Time for Entire Process: {normalized_total_time:.8f} seconds per MB")
print(f"Total Average CPU Usage for Entire Process: {total_avg_cpu_usage:.2f}%")

# Display the metrics DataFrame
metrics_df


Total Execution Time for Entire Process (Raw): 19 minutes and 14.60 seconds
Total Normalized Execution Time for Entire Process: 1.00777570 seconds per MB
Total Average CPU Usage for Entire Process: 70.83%


,RMSE,MAPE,R2,MSE,nRMSE,Execution Time (Raw),Normalized Time (s/MB),Average CPU Usage,Sample Size
0,0.416879,0.107996,0.182581,0.173788,0.138960,665.410578,1.098601,50.50,1.000
1,0.413897,0.104905,0.177785,0.171311,0.137966,262.668047,0.866469,54.15,0.500
2,0.414252,0.104988,0.176376,0.171604,0.138084,130.584456,0.861523,78.15,0.250
3,0.415081,0.105257,0.173074,0.172292,0.138360,68.812837,0.907977,81.55,0.125
4,0.481996,0.120673,0.106460,0.232320,0.160665,0.666103,77.953097,82.45,100.000
5,0.460827,0.109774,0.047598,0.212362,0.153609,3.999035,46.800131,75.50,1000.000
6,0.431509,0.120512,0.131600,0.186200,0.143836,8.473391,9.916289,73.25,10000.000
7,0.417229,0.107568,0.149344,0.174080,0.139076,13.985374,1.636688,71.10,100000.000
